In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint
# from config import api_key
import numpy as np
import os
import tensorflow as tf

In [5]:
scorecard = pd.read_csv("resources/Scorecard.csv")
scorecard

FileNotFoundError: [Errno 2] No such file or directory: 'resources/Scorecard.csv'

In [ ]:
# scorecard_reduced = scorecard[["INSTNM", "CITY", "ZIP", "NUMBRANCH", "HIGHDEG", "CONTROL", "st_fips", "region", 
#                                "ADM_RATE_ALL", "SATVRMID", "SATMTMID", "SAT_AVG", "PCIP01", 
#                                "PCIP03", "PCIP04", "PCIP05", "PCIP09", "PCIP10", "PCIP11", "PCIP12", "PCIP13", "PCIP14",
#                                "PCIP15", "PCIP16", "PCIP19", "PCIP22", "PCIP23", "PCIP24", "PCIP25", "PCIP26", "PCIP27",
#                                "PCIP29", "PCIP30", "PCIP31", "PCIP38", "PCIP39", "PCIP40", "PCIP41", "PCIP42", "PCIP43",
#                                "PCIP44", "PCIP45", "PCIP46", "PCIP47", "PCIP48", "PCIP49", "PCIP50", "PCIP51", "PCIP52",
#                                "PCIP54", "UGDS", "CURROPER", "TUITIONFEE_IN", "TUITIONFEE_OUT", "INEXPFTE", "AVGFACSAL",
#                                "C150_4", "D150_4", "CDR2", "DEBT_MDN", "CUML_DEBT_N", "INC_N", "APPL_SCH_N", "REPAY_DT_N", 
#                                "gt_25k_p10"]]

In [ ]:
scorecard_cleaned = scorecard[["INSTNM", "NUMBRANCH", "HIGHDEG", "CONTROL", "st_fips", "region", 
                               "ADM_RATE_ALL", "SAT_AVG", "UGDS", "CURROPER", "TUITIONFEE_IN", "TUITIONFEE_OUT", "INEXPFTE", "AVGFACSAL",
                               "C150_4", "D150_4", "CDR2", "DEBT_MDN", "CUML_DEBT_N", "INC_N", "APPL_SCH_N", "REPAY_DT_MDN", "faminc", "age_entry",
                               "mn_earn_wne_p6"]]
scorecard_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124699 entries, 0 to 124698
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   INSTNM          124699 non-null  object 
 1   NUMBRANCH       124699 non-null  int64  
 2   HIGHDEG         124699 non-null  object 
 3   CONTROL         124676 non-null  object 
 4   st_fips         124669 non-null  object 
 5   region          124669 non-null  object 
 6   ADM_RATE_ALL    37385 non-null   float64
 7   SAT_AVG         18598 non-null   float64
 8   UGDS            110343 non-null  float64
 9   CURROPER        124699 non-null  object 
 10  TUITIONFEE_IN   58024 non-null   float64
 11  TUITIONFEE_OUT  56844 non-null   float64
 12  INEXPFTE        106049 non-null  float64
 13  AVGFACSAL       69652 non-null   float64
 14  C150_4          34880 non-null   float64
 15  D150_4          34880 non-null   float64
 16  CDR2            111601 non-null  float64
 17  DEBT_MDN  

In [ ]:
rep_priv = scorecard_cleaned.replace("PrivacySuppressed", "")

In [ ]:
#the dataset has 25k as the "threshold earnings" and shows a percentage of people from different universities meeting the threshold, so this will be target
cardscores = rep_priv.dropna(subset=['mn_earn_wne_p6'])

In [ ]:
cardscores

,INSTNM,NUMBRANCH,HIGHDEG,CONTROL,st_fips,region,ADM_RATE_ALL,SAT_AVG,UGDS,CURROPER,...,AVGFACSAL,C150_4,D150_4,CDR2,DEBT_MDN,CUML_DEBT_N,INC_N,APPL_SCH_N,REPAY_DT_MDN,mn_earn_wne_p6
46082,ALABAMA A & M UNIVERSITY,1,Graduate degree,Public,Alabama,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",0.4457,850.0,5292.0,Currently certified as operating,...,4991.0,0.3802,789.0,0.124,13187,2795,,2862,17132,30300
46083,UNIVERSITY OF ALABAMA AT BIRMINGHAM,1,Graduate degree,Public,Alabama,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",0.8049,1010.0,10381.0,Currently certified as operating,...,6734.0,0.3857,1115.0,0.037,10500,4533,5005,5022,17111,37500
46084,SOUTHERN CHRISTIAN UNIVERSITY,1,Graduate degree,Private nonprofit,Alabama,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",NaN,NaN,336.0,Currently certified as operating,...,3043.0,NaN,NaN,0.028,10158,272,,312,16962,
46085,UNIVERSITY OF ALABAMA IN HUNTSVILLE,1,Graduate degree,Public,Alabama,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",0.8756,1130.0,4966.0,Currently certified as operating,...,6257.0,0.4369,396.0,0.047,10500,1962,,1939,17174,40600
46086,ALABAMA STATE UNIVERSITY,1,Graduate degree,Public,Alabama,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",0.5042,NaN,4941.0,Currently certified as operating,...,5017.0,0.2117,954.0,0.179,9938,3864,3295,3308,17198,25300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109092,Strayer University-Charleston Campus,84,Non-degree-granting,Private for-profit,South Carolina,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",NaN,NaN,NaN,Currently certified as operating,...,NaN,NaN,NaN,0.085,12667,48621,31416,31416,19305,44200
109093,Strayer University-Irving,84,Non-degree-granting,Private for-profit,Texas,"Southwest (AZ, NM, OK, TX)",NaN,NaN,NaN,Currently certified as operating,...,NaN,NaN,NaN,0.085,12667,48621,31416,31416,19305,44200
109094,Strayer University-Katy,84,Non-degree-granting,Private for-profit,Texas,"Southwest (AZ, NM, OK, TX)",NaN,NaN,NaN,Currently certified as operating,...,NaN,NaN,NaN,0.085,12667,48621,31416,31416,19305,44200
109095,Strayer University-Northwest Houston,84,Non-degree-granting,Private for-profit,Texas,"Southwest (AZ, NM, OK, TX)",NaN,NaN,NaN,Currently certified as operating,...,NaN,NaN,NaN,0.085,12667,48621,31416,31416,19305,44200


In [ ]:
cardscores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32688 entries, 46082 to 109096
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   INSTNM          32688 non-null  object 
 1   NUMBRANCH       32688 non-null  int64  
 2   HIGHDEG         32688 non-null  object 
 3   CONTROL         32688 non-null  object 
 4   st_fips         32687 non-null  object 
 5   region          32687 non-null  object 
 6   ADM_RATE_ALL    14487 non-null  float64
 7   SAT_AVG         7235 non-null   float64
 8   UGDS            30874 non-null  float64
 9   CURROPER        32688 non-null  object 
 10  TUITIONFEE_IN   20678 non-null  float64
 11  TUITIONFEE_OUT  20333 non-null  float64
 12  INEXPFTE        31413 non-null  float64
 13  AVGFACSAL       20707 non-null  float64
 14  C150_4          10947 non-null  float64
 15  D150_4          10947 non-null  float64
 16  CDR2            31677 non-null  float64
 17  DEBT_MDN        29617 non-null 

In [ ]:
columns_to_convert = ['DEBT_MDN', 'CUML_DEBT_N', 'INC_N', 
                      'APPL_SCH_N', 'REPAY_DT_MDN', 'mn_earn_wne_p6']
cardscores[columns_to_convert] = cardscores[columns_to_convert].replace('', np.nan)

cardscores[columns_to_convert] = cardscores[columns_to_convert].astype(float)

C:\Users\armat\AppData\Local\Temp\ipykernel_15744\913918454.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardscores[columns_to_convert] = cardscores[columns_to_convert].replace('', np.nan)
C:\Users\armat\AppData\Local\Temp\ipykernel_15744\913918454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardscores[columns_to_convert] = cardscores[columns_to_convert].astype(float)


In [ ]:
cardscores.columns

Index(['INSTNM', 'NUMBRANCH', 'HIGHDEG', 'CONTROL', 'st_fips', 'region',
       'ADM_RATE_ALL', 'SAT_AVG', 'UGDS', 'CURROPER', 'TUITIONFEE_IN',
       'TUITIONFEE_OUT', 'INEXPFTE', 'AVGFACSAL', 'C150_4', 'D150_4', 'CDR2',
       'DEBT_MDN', 'CUML_DEBT_N', 'INC_N', 'APPL_SCH_N', 'REPAY_DT_MDN',
       'mn_earn_wne_p6'],
      dtype='object')

In [ ]:
for col in cardscores.columns:
    if cardscores[col].dtype in ['float64', 'int64']:
        mean_value = cardscores[col].mean()
        cardscores[col].fillna(mean_value, inplace=True)


C:\Users\armat\AppData\Local\Temp\ipykernel_15744\239783290.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cardscores[col].fillna(mean_value, inplace=True)
C:\Users\armat\AppData\Local\Temp\ipykernel_15744\239783290.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardscores[col].fillna(mean_value, inplace=True)


In [ ]:
# cardscores.to_csv("test3.csv")

In [ ]:
#converting float to int here to reduce memory demands for model training
# floaters = cardscores.select_dtypes(include=['float']).columns
# float_columns = [col for col in floaters if col != "gt_25k_p10"]
# cardscores[float_columns] = cardscores[float_columns].astype(int)
# cardscores["gt_25k_p10"] = cardscores["gt_25k_p10"].astype(float)

In [ ]:
cardscores.to_csv("cards.csv")

In [ ]:
cardscores.sort_values(by="mn_earn_wne_p6", ascending=False).head(12)

,INSTNM,NUMBRANCH,HIGHDEG,CONTROL,st_fips,region,ADM_RATE_ALL,SAT_AVG,UGDS,CURROPER,...,AVGFACSAL,C150_4,D150_4,CDR2,DEBT_MDN,CUML_DEBT_N,INC_N,APPL_SCH_N,REPAY_DT_MDN,mn_earn_wne_p6
105721,Medical College of Wisconsin,1,Graduate degree,Private nonprofit,Wisconsin,"Great Lakes (IL, IN, MI, OH, WI)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,5425.378761,0.486122,628.019092,0.000,9337.004446,6314.339502,8122.084872,6370.447796,29667.530274,151000.0
90130,Oklahoma State University Center for Health Sc...,1,Graduate degree,Public,Oklahoma,"Southwest (AZ, NM, OK, TX)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,5425.378761,0.486122,628.019092,0.000,6625.000000,6314.339502,8122.084872,6370.447796,29667.530274,142800.0
89069,A T Still University of Health Sciences,1,Graduate degree,Private nonprofit,Missouri,"Plains (IA, KS, MN, MO, NE, ND, SD)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,7586.000000,0.486122,628.019092,0.002,37160.000000,6314.339502,8122.084872,6370.447796,29667.530274,124200.0
73303,Western University of Health Sciences,1,Graduate degree,Private nonprofit,California,"Far West (AK, CA, HI, NV, OR, WA)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,8283.000000,0.486122,628.019092,0.000,9337.004446,6314.339502,158.000000,158.000000,18018.000000,122400.0
91234,Medical College of Wisconsin,1,Graduate degree,Private nonprofit,Wisconsin,"Great Lakes (IL, IN, MI, OH, WI)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,5425.378761,0.486122,628.019092,0.000,4775.000000,6314.339502,8122.084872,6370.447796,29667.530274,122100.0
73069,Thunderbird School of Global Management,1,Graduate degree,Private nonprofit,Arizona,"Southwest (AZ, NM, OK, TX)",0.695113,1052.416171,2394.568666,Not currently certified as an operating instit...,...,9766.000000,0.486122,628.019092,0.008,9337.004446,6314.339502,126.000000,126.000000,18141.000000,117700.0
73235,University of California Hastings College of Law,1,Graduate degree,Public,California,"Far West (AK, CA, HI, NV, OR, WA)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,13995.000000,0.486122,628.019092,0.001,9337.004446,6314.339502,357.000000,357.000000,29667.530274,117400.0
59571,University of California Hastings College of Law,1,Graduate degree,Public,California,"Far West (AK, CA, HI, NV, OR, WA)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,11581.000000,0.486122,628.019092,0.004,14700.000000,6314.339502,409.000000,409.000000,29667.530274,117000.0
75600,Brooklyn Law School,1,Graduate degree,Private nonprofit,New York,"Mid East (DE, DC, MD, NJ, NY, PA)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,15528.000000,0.486122,628.019092,0.000,5000.000000,125.000000,8122.084872,345.000000,18353.000000,115900.0
59642,Western University of Health Sciences,1,Graduate degree,Private nonprofit,California,"Far West (AK, CA, HI, NV, OR, WA)",0.695113,1052.416171,2394.568666,Currently certified as operating,...,7759.000000,0.486122,628.019092,0.000,9337.004446,14.000000,150.000000,150.000000,17653.000000,114500.0


In [ ]:
cardscores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32688 entries, 46082 to 109096
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NUMBRANCH         32688 non-null  int64  
 1   HIGHDEG           32688 non-null  object 
 2   CONTROL           32688 non-null  object 
 3   st_fips           32687 non-null  object 
 4   region            32687 non-null  object 
 5   ADM_RATE_ALL      32688 non-null  float64
 6   SAT_AVG           32688 non-null  float64
 7   UGDS              32688 non-null  float64
 8   CURROPER          32688 non-null  object 
 9   TUITIONFEE_IN     32688 non-null  float64
 10  TUITIONFEE_OUT    32688 non-null  float64
 11  INEXPFTE          32688 non-null  float64
 12  AVGFACSAL         32688 non-null  float64
 13  C150_4            32688 non-null  float64
 14  D150_4            32688 non-null  float64
 15  CDR2              32688 non-null  float64
 16  DEBT_MDN          32688 non-null  float6

In [ ]:
# cardscores['relative_success'] = cardscores['gt_25k_p10'].apply(classify_relative_success)
cardscores.loc[:, 'relative_success'] = np.where(cardscores['mn_earn_wne_p6'] > 30000, "yes", "no")

C:\Users\armat\AppData\Local\Temp\ipykernel_15744\791218331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardscores.loc[:, 'relative_success'] = np.where(cardscores['mn_earn_wne_p6'] > 30000, "yes", "no")


In [ ]:
cardscores.drop("INSTNM", axis=1, inplace=True)

C:\Users\armat\AppData\Local\Temp\ipykernel_15744\740589651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardscores.drop("INSTNM", axis=1, inplace=True)


In [ ]:
numeric_scorecard = pd.get_dummies(cardscores)
# numeric_scorecard

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

MODEL 1

In [ ]:
X = numeric_scorecard.drop(columns=["relative_success_no", "relative_success_yes", "mn_earn_wne_p6"], axis=1)
y = numeric_scorecard["relative_success_no"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=y)

In [ ]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
input_features = X_train_scaled.shape[1]
hidden_nodes_1 = 30
hidden_nodes_2 = 25
hidden_nodes_3 = 15

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, input_dim = input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.summary()

c:\Users\armat\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_96 (Dense)                │ (None, 30)             │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_97 (Dense)                │ (None, 25)             │           775 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_98 (Dense)                │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,061 (15.86 KB)

 Trainable params: 4,061 (15.86 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model = nn.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 2s 785us/step - accuracy: 0.6892 - loss: 0.5725
Epoch 2/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 747us/step - accuracy: 0.7722 - loss: 0.4647
Epoch 3/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 817us/step - accuracy: 0.7826 - loss: 0.4460
Epoch 4/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 759us/step - accuracy: 0.7883 - loss: 0.4350
Epoch 5/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 765us/step - accuracy: 0.7989 - loss: 0.4157
Epoch 6/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.7980 - loss: 0.4146
Epoch 7/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - accuracy: 0.7982 - loss: 0.4092
Epoch 8/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step - accuracy: 0.8102 - loss: 0.3961
Epoch 9/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 792us/step - accuracy: 0.8169 - loss: 0.3882
Epoch 10/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 842us/step - accuracy: 0.8119 - loss: 0.3936
Epoch 11/25
767/767 ━━━━━━━━━━━━━━━━━━━━ 1s 804us/step - accuracy: 0.8132 - loss: 0.3856
Epoch 12/25
767/767 ━━━━━━━━━━

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

256/256 - 0s - 922us/step - accuracy: 0.8041 - loss: 0.4213
Loss: 0.4212554693222046, Accuracy: 0.8040871024131775


Model 2: Checking for data leakage

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
lin_regress = LinearRegression()
# sample = numeric_scorecard.sample(frac=0.3, random_state=42)
X = numeric_scorecard.drop(columns=["relative_success_no", "relative_success_yes", "mn_earn_wne_p6"], axis=1)
y = numeric_scorecard["relative_success_no"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=28)

In [ ]:
lin_regress.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = lin_regress.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 0.12
R-squared: 0.35


In [ ]:
me = mse ** .5
print(me)

0.4258975423666139


Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(mse)
print(r2)

0.09928202398433676
0.5982555255741309


In [ ]:
importances = rf.feature_importances_

In [ ]:
feature_names = X.columns
feature_importance_list = [(feature, importance) for feature, importance in zip(feature_names, importances)]
feature_importance_list = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)

In [ ]:
for feature, importance in feature_importance_list:
    print(f"Feature: {feature}, Importance: {importance}")

Feature: HIGHDEG_Graduate degree, Importance: 0.16496413487672953
Feature: APPL_SCH_N, Importance: 0.10774766627462848
Feature: DEBT_MDN, Importance: 0.0932826258161714
Feature: CDR2, Importance: 0.07200960703419813
Feature: REPAY_DT_MDN, Importance: 0.06415420924815686
Feature: INEXPFTE, Importance: 0.05427767396680853
Feature: UGDS, Importance: 0.05075925888743757
Feature: AVGFACSAL, Importance: 0.04615191807015208
Feature: CUML_DEBT_N, Importance: 0.0450242736491571
Feature: TUITIONFEE_OUT, Importance: 0.03601419852687077
Feature: INC_N, Importance: 0.030885786073069025
Feature: TUITIONFEE_IN, Importance: 0.028694940698401138
Feature: ADM_RATE_ALL, Importance: 0.021472692030106377
Feature: NUMBRANCH, Importance: 0.016402864737456363
Feature: D150_4, Importance: 0.015395338052718947
Feature: CONTROL_Private for-profit, Importance: 0.010920812059791665
Feature: C150_4, Importance: 0.009441303815095729
Feature: region_Outlying Areas (AS, FM, GU, MH, MP, PR, PW, VI), Importance: 0.00811

Re-modeling with 10 year post grad earnings vs 6 year

In [ ]:
scorecard_cleaner = scorecard[["INSTNM", "NUMBRANCH", "HIGHDEG", "CONTROL", "st_fips", "region", 
                               "ADM_RATE_ALL", "SATVRMID", "SATMTMID", "SAT_AVG", "UGDS", "CURROPER", "TUITIONFEE_IN", "TUITIONFEE_OUT", "INEXPFTE", "AVGFACSAL",
                               "C150_4", "D150_4", "CDR2", "DEBT_MDN", "CUML_DEBT_N", "INC_N", "APPL_SCH_N", "REPAY_DT_MDN", 
                               "mn_earn_wne_p10"]]

In [ ]:
rep_priv = scorecard_cleaner.replace("PrivacySuppressed", "")

In [ ]:
card_scores_10yr = rep_priv.dropna(subset=["mn_earn_wne_p10"])

In [ ]:
columns_to_convert = ['DEBT_MDN', 'CUML_DEBT_N', 'INC_N', 
                      'APPL_SCH_N', 'REPAY_DT_MDN', 'mn_earn_wne_p10']
card_scores_10yr[columns_to_convert] = card_scores_10yr[columns_to_convert].replace('', np.nan)

card_scores_10yr[columns_to_convert] = card_scores_10yr[columns_to_convert].astype(float)

C:\Users\armat\AppData\Local\Temp\ipykernel_15744\1974156169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  card_scores_10yr[columns_to_convert] = card_scores_10yr[columns_to_convert].replace('', np.nan)
C:\Users\armat\AppData\Local\Temp\ipykernel_15744\1974156169.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  card_scores_10yr[columns_to_convert] = card_scores_10yr[columns_to_convert].astype(float)


In [ ]:
for col in cardscores.columns:
    if cardscores[col].dtype in ['float64', 'int64']:
        mean_value = cardscores[col].mean()
        cardscores[col].fillna(mean_value, inplace=True)

C:\Users\armat\AppData\Local\Temp\ipykernel_15744\1321557762.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cardscores[col].fillna(mean_value, inplace=True)


In [ ]:
card_scores_10yr.loc[:, 'relative_successes'] = np.where(
    (card_scores_10yr['mn_earn_wne_p10'] > 60000) & (card_scores_10yr['DEBT_MDN'] < 15000), 
    "yes", 
    "no"
)

In [ ]:
card_scores_10yr.drop("INSTNM", axis=1, inplace=True)

KeyError: "['INSTNM'] not found in axis"

In [ ]:
numeric_scorecards = pd.get_dummies(card_scores_10yr)
numeric_scorecards

,NUMBRANCH,ADM_RATE_ALL,SATVRMID,SATMTMID,SAT_AVG,UGDS,TUITIONFEE_IN,TUITIONFEE_OUT,INEXPFTE,AVGFACSAL,...,"region_Outlying Areas (AS, FM, GU, MH, MP, PR, PW, VI)","region_Plains (IA, KS, MN, MO, NE, ND, SD)","region_Rocky Mountains (CO, ID, MT, UT, WY)","region_Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC, TN, VA, WV)","region_Southwest (AZ, NM, OK, TX)",region_U.S. Service Schools,CURROPER_Currently certified as operating,CURROPER_Not currently certified as an operating institution,relative_successes_no,relative_successes_yes
72999,1,0.3182,NaN,NaN,850.0,4704.0,4072.0,7304.0,4804.0,6001.0,...,False,False,False,True,False,False,True,False,True,False
73000,1,0.8778,545.0,575.0,1106.0,10381.0,4208.0,9296.0,15733.0,8697.0,...,False,False,False,True,False,False,True,False,True,False
73001,1,NaN,NaN,NaN,NaN,343.0,6240.0,6240.0,5532.0,3217.0,...,False,False,False,True,False,False,True,False,True,False
73002,1,0.8800,550.0,555.0,1121.0,5562.0,5216.0,11024.0,7548.0,7777.0,...,False,False,False,True,False,False,True,False,True,False
73003,1,0.4159,405.0,390.0,784.0,4647.0,4508.0,8516.0,5740.0,6430.0,...,False,False,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109092,84,NaN,NaN,NaN,NaN,NaN,14805.0,NaN,NaN,NaN,...,False,False,False,True,False,False,True,False,True,False
109093,84,NaN,NaN,NaN,NaN,NaN,14805.0,NaN,NaN,NaN,...,False,False,False,False,True,False,True,False,True,False
109094,84,NaN,NaN,NaN,NaN,NaN,14805.0,NaN,NaN,NaN,...,False,False,False,False,True,False,True,False,True,False
109095,84,NaN,NaN,NaN,NaN,NaN,14805.0,NaN,NaN,NaN,...,False,False,False,False,True,False,True,False,True,False


In [ ]:
X = numeric_scorecards.drop(columns=["relative_successes_no", "relative_successes_yes", "mn_earn_wne_p10", "DEBT_MDN"], axis=1)
y = numeric_scorecards["relative_successes_no"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=y)

In [ ]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
input_features = X_train_scaled.shape[1]
hidden_nodes_1 = 30
hidden_nodes_2 = 25
hidden_nodes_3 = 15

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, input_dim = input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.summary()

c:\Users\armat\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_92 (Dense)                │ (None, 30)             │         2,910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_93 (Dense)                │ (None, 25)             │           775 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_94 (Dense)                │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_95 (Dense)                │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,091 (15.98 KB)

 Trainable params: 4,091 (15.98 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model = nn.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25


453/453 ━━━━━━━━━━━━━━━━━━━━ 1s 835us/step - accuracy: 0.9663 - loss: 0.4222
Epoch 2/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.9801 - loss: 0.0979
Epoch 3/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.9787 - loss: 0.1034
Epoch 4/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - accuracy: 0.9798 - loss: 0.0988
Epoch 5/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - accuracy: 0.9804 - loss: 0.0967
Epoch 6/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.9806 - loss: 0.0957
Epoch 7/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - accuracy: 0.9795 - loss: 0.1000
Epoch 8/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.9788 - loss: 0.1028
Epoch 9/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.9797 - loss: 0.0993
Epoch 10/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.9796 - loss: 0.0996
Epoch 11/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.9783 - loss: 0.1046
Epoch 12/25
453/453 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

151/151 - 0s - 1ms/step - accuracy: 0.9795 - loss: 0.1000
Loss: 0.10001175850629807, Accuracy: 0.9794946312904358


In [ ]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
importances = rf.feature_importances_
feature_names = X.columns
feature_importance_list = [(feature, importance) for feature, importance in zip(feature_names, importances)]
feature_importance_list = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)
for feature, importance in feature_importance_list:
    print(f"Feature: {feature}, Importance: {importance}")

Feature: APPL_SCH_N, Importance: 0.26105580449106536
Feature: INC_N, Importance: 0.17627998652607008
Feature: CUML_DEBT_N, Importance: 0.05733505905176826
Feature: INEXPFTE, Importance: 0.04791194607978
Feature: REPAY_DT_MDN, Importance: 0.047389151150438916
Feature: UGDS, Importance: 0.04163262952301229
Feature: CDR2, Importance: 0.04138041936075311
Feature: ADM_RATE_ALL, Importance: 0.03818398541975253
Feature: AVGFACSAL, Importance: 0.03569460598857238
Feature: SAT_AVG, Importance: 0.03474515944219709
Feature: TUITIONFEE_IN, Importance: 0.027142893206198615
Feature: C150_4, Importance: 0.02573325837785988
Feature: TUITIONFEE_OUT, Importance: 0.016858722249182785
Feature: D150_4, Importance: 0.016290799510710965
Feature: SATVRMID, Importance: 0.014331985620130737
Feature: SATMTMID, Importance: 0.012377540575256824
Feature: NUMBRANCH, Importance: 0.011515478960462004
Feature: region_U.S. Service Schools, Importance: 0.008605756768293973
Feature: CONTROL_Public, Importance: 0.007385131